In [ ]:
import sys
import os
import glob
import json

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, os.path.join(project_root, 'src'))

import chunking.chunker as chunk
import embedding.embedder as emb
import loaders.loaders as load
import loaders.proccesing as proc
#import retrievers.retrievers as ret
import vector_store_client.vector_client as vsc
# import pdf_processing.pdf_to_json as pdf
# import text_preprocessing.text_preprocessing as tp

In [ ]:
input_path = "../data/reglamentacion/"
output_path = "../data/staging/"

In [ ]:
processor = proc.Processing(ruta=input_path, output=output_path)

# Convert PDFs to JSON
processor.pdf_to_json()

# Clean the JSON data
processor.clean_data()

In [ ]:
all_chunks = []

for file in glob.glob(output_path+'*.json'):
    # Load a cleaned JSON file
    print(file)
    with open(file, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Pass it to Chunker
    chunker = chunk.Chunker(data, max_chunk_size=600)
    chunks = chunker.chunking_hybrid()

    all_chunks.extend(chunks)

In [ ]:
print(len(all_chunks))
all_chunks[0]

In [ ]:
embedder = emb.Embedder()
enriched_chunks = embedder.generate_embeddings(chunks)

In [ ]:
len(enriched_chunks)

In [ ]:
vector_client = vsc.VectorStoreClient(type="qdrant")

# Insert embeddings into the vector store
vector_client.insert_embeddings(enriched_chunks)

In [ ]:

# Perform a search
query = "Sample search query"
results = vector_client.search(query)

# Display search results
for result in results:
    print(result)